# Comparison Between DASH and ACTR
See the rest here: https://github.com/DannyWeitekamp/HLAHTOI/tree/master/assignment1

In [33]:
import numpy as np
import pandas as pd

In [34]:
def gen_learner_history(now,times,correct,alpha=0.0,delta=0.0,windows=[1/24,1,7,30]):
    times = np.array(times)
    correct = np.array(correct)
    elapse = now-times
    windows = [0.0] + [86400.0*x for x in windows] + [float('inf')]
    lh = {}
    for i in range(len(windows)-1):
        upper = windows[i+1]
        lower = windows[i]
        in_win = np.logical_and(elapse > lower,elapse <= upper)
        c_in_win = in_win * correct
        
        lh['n_w'+str(i)] = in_win.sum()
        lh['c_w'+str(i)] = c_in_win.sum()
    lh["elapses"] = elapse
    lh["alpha"] = alpha
    lh["delta"] = delta
    return lh

def day(x):
    return x * 86400.0
def hour(x):
    return x * 3600
def mins(x):
    return x * 60



# Fitting The DASH Model

Because DASH doesn't come with a description of parameter values, I had to fit a model to data. I used some data that Theo used in his study; preprocessed data from the assistments17 dataset. See the other two notebooks in this github for how I processed the data further for this purpose. I tried fitting the data with pyTorch, but it didn't converge, so I tried again by using with the glmer package in R (this one worked better). I made a phi and psi fixed effects and had random effects for the student and kc intercepts (I also did not use a global intercept). I include the fit parameters for both models here.

In [35]:
#---Fit with torch---
#phi tensor([-0.1015,  0.1419,  0.1439,  0.1933,  0.0000])
#psi tensor([-0.1941,  0.1973,  0.0245,  0.0347,  0.0000])

#---Fit with glmer (in R)----
#    ltc_0      lop_0      ltc_1      lop_1      ltc_2      lop_2      ltc_3      lop_3      ltc_4    lop_4  
#  0.038803  -0.011770   0.041937  -0.029086   0.060719  -0.038129   0.033654  -0.028097   0.010040  -0.004177  
    
 
window_profiles = [{
    'scale' : 1/24,
    'phi' :  0.038803,
    'psi' : -0.011770
},
{
    'scale' : 1,
    'phi' :  0.041937,
    'psi' : -0.029086
},
{
    'scale' : 7,
    'phi' : 0.060719,
    'psi' : -0.038129
},
{
    'scale' : 30,
    'phi' : 0.033654,
    'psi' : -0.028097
},
]

def sigmoid(x):
    return 1/(1 + np.exp(-x)) 

def DASH_activation(lh,wps):
    s = 0
    for i,wp in enumerate(wps):
        s += wp["phi"]*np.log(1+lh["c_w"+str(i)])-wp["psi"]*np.log(1+lh["n_w"+str(i)])
    #print(s*10)
    s*=10
    return lh['alpha'] - lh['delta'] + s

def DASH(lh,wps):
    x = DASH_activation(lh,wps)
    return 1.0/(1 + np.exp(-x)) 

def whitehill_activation(lh):
    B_t = 0#(lh['alpha'] - lh['delta']) #Best Guess
    return np.log(np.sum(np.power(lh['elapses']/3600.0,-.5)))+B_t

def whitehill(lh):
    x = whitehill_activation(lh)
    return 1.0/(1 + np.exp(-x)) 

# Comparison Between DASH and Whitehill Implementation of(ACT-R)

I compare predictions of DASH vs Whitehill's ACT-R implementation on three different studying patterns:
1. Crammed: A little bit of study 2 days before, lots of study 30 minutes before the test
2. Spaced: Study sessions are spaced over different days
3. Massed: Lots of Study 2 days before the test


In [36]:
correctness = [0,0,1,0,0,1,1,0,1,1,1]
massed_short_h = [
    day(12)+hour(7)+mins(50),
    day(12)+hour(7)+mins(51),
    day(12)+hour(7)+mins(52),
    day(12)+hour(7)+mins(53),
    day(12)+hour(7)+mins(54),
    day(12)+hour(7)+mins(55),
    day(12)+hour(7)+mins(56),
    day(12)+hour(7)+mins(57),
    day(12)+hour(7)+mins(58),
    day(12)+hour(7)+mins(59),
    day(12)+hour(8)+mins(0),
]

massed_long_h = [
    day(11)+hour(7)+mins(50),
    day(11)+hour(7)+mins(51),
    day(11)+hour(7)+mins(52),
    day(11)+hour(7)+mins(53),
    day(11)+hour(7)+mins(54),
    day(11)+hour(7)+mins(55),
    day(11)+hour(7)+mins(56),
    day(11)+hour(7)+mins(57),
    day(11)+hour(7)+mins(58),
    day(11)+hour(7)+mins(59),
    day(11)+hour(8)+mins(0),
]



spaced_short_h = [
    day(2)+hour(0)+mins(0),
    day(3)+hour(0)+mins(0),
    day(4)+hour(0)+mins(0),
    day(5)+hour(0)+mins(0),
    day(6)+hour(0)+mins(0),
    day(7)+hour(0)+mins(0),
    day(8)+hour(0)+mins(0),
    day(9)+hour(0)+mins(0),
    day(10)+hour(0)+mins(0),
    day(11)+hour(0)+mins(0),
    day(12)+hour(0)+mins(0),
]

spaced_long_h = [
    day(1)+hour(0)+mins(0),
    day(2)+hour(0)+mins(0),
    day(3)+hour(0)+mins(0),
    day(4)+hour(0)+mins(0),
    day(5)+hour(0)+mins(0),
    day(6)+hour(0)+mins(0),
    day(7)+hour(0)+mins(0),
    day(8)+hour(0)+mins(0),
    day(9)+hour(0)+mins(0),
    day(10)+hour(0)+mins(0),
    day(11)+hour(0)+mins(0),
]





# Results

Interestingly ACT-R with constant decay predicts that massing is the best, but DASH predicts that spacing is the best. Values are printed out here as the chance the student gets the test item correct (from 0% to 100%).

In [37]:
now = day(12)+hour(8)+mins(10)
masses_short_lh =  gen_learner_history(now, massed_short_h, correctness,alpha=.05,delta=.1)
massed_long_lh  =  gen_learner_history(now, massed_long_h,  correctness,alpha=.05,delta=.1)
spaced_short_lh =  gen_learner_history(now, spaced_short_h, correctness,alpha=.05,delta=.1) 
spaced_long_lh  =  gen_learner_history(now, spaced_long_h,  correctness,alpha=.05,delta=.1) 

print("DASH Predictions")
print("Massed-Short-Retention\t: %2.2f%%" % (100 * DASH(masses_short_lh,window_profiles)))
print("Massed-Long-Retention\t: %2.2f%%" % (100 * DASH(massed_long_lh,window_profiles)))
print("Spaced-Short-Retention\t: %2.2f%%" % (100 * DASH(spaced_short_lh,window_profiles)))
print("Spaced-Long-Retention\t: %2.2f%%" % (100 * DASH(spaced_long_lh,window_profiles)))
print("---------------------------------")
print("Whitehill ACT-R (constant decay) Predictions")
print("Massed-Short-Retention\t: %2.2f%%" % (100 * whitehill(masses_short_lh)))
print("Massed-Long-Retention\t: %2.2f%%" % (100 * whitehill(massed_long_lh)))
print("Spaced-Short-Retention\t: %2.2f%%" % (100 * whitehill(spaced_short_lh)))
print("Spaced-Long-Retention\t: %2.2f%%" % (100 * whitehill(spaced_long_lh)))

DASH Predictions
Massed-Short-Retention	: 73.06%
Massed-Long-Retention	: 88.88%
Spaced-Short-Retention	: 94.52%
Spaced-Long-Retention	: 92.53%
---------------------------------
Whitehill ACT-R (constant decay) Predictions
Massed-Short-Retention	: 95.72%
Massed-Long-Retention	: 69.08%
Spaced-Short-Retention	: 56.80%
Spaced-Long-Retention	: 50.63%
